# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config_zack import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("../output_data/cities.csv")
city_data_df.head


<bound method NDFrame.head of      City_ID      city  Latitude  Longitude  Max Temperature  Humidity  \
0          0    dikson   73.5069    80.5464            42.69        87   
1          1  hermanus  -34.4187    19.2345            49.89        85   
2          2     alofi  -19.0595  -169.9187            75.09        83   
3          3  flinders  -34.5833   150.8552            66.18        39   
4          4   almeria   37.1667    -2.3333            85.14        52   
..       ...       ...       ...        ...              ...       ...   
548      548     natal   -5.7950   -35.2094            74.05       100   
549      549      tank   32.2217    70.3793            93.45        35   
550      550     perth  -31.9333   115.8333            56.48        90   
551      551     lamar   33.6668   -95.5836            91.06        64   
552      552  ambilobe  -13.2000    49.0500            67.48        81   

     Cloudiness  Wind Speed Country        Date  
0            32        9.89    

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#Heatmap of humidity
locations = city_data_df[["Latitude", "Longitude"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
humid_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=400, point_radius=1)

fig.add_layer(humid_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_city_df = city_data_df.loc[(city_data_df["Max Temperature"] < 75) & (city_data_df["Max Temperature"] > 65) & (city_data_df["Wind Speed"] < 15) & (city_data_df["Humidity"] < 35)].dropna()

ideal_city_df

,City_ID,city,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
76,76,ituiutaba,-18.9689,-49.4650,71.19,25,11,3.49,BR,1628730576
182,182,lucapa,-8.4192,20.7447,65.75,27,0,3.49,AO,1628730600
278,278,mandalgovi,45.7625,106.2708,71.56,27,58,4.18,MN,1628730487
283,283,komsomolskiy,40.4272,71.7189,72.82,25,0,5.50,UZ,1628730627
420,420,mount isa,-20.7333,139.5000,74.97,22,0,10.36,AU,1628730658
448,448,shahrud,36.4182,54.9763,73.42,21,0,2.57,IR,1628730665
485,485,barra do garcas,-15.8900,-52.2567,74.35,30,0,2.30,BR,1628730674
493,493,srednekolymsk,67.4500,153.6833,70.09,29,100,0.78,RU,1628730675


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Create DataFrame hotel_df to store data
hotel_df = ideal_city_df[["city", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,city,Country,Latitude,Longitude,Hotel Name
76,ituiutaba,BR,-18.9689,-49.4650,
182,lucapa,AO,-8.4192,20.7447,
278,mandalgovi,MN,45.7625,106.2708,
283,komsomolskiy,UZ,40.4272,71.7189,
420,mount isa,AU,-20.7333,139.5000,
448,shahrud,IR,36.4182,54.9763,
485,barra do garcas,BR,-15.8900,-52.2567,
493,srednekolymsk,RU,67.4500,153.6833,


In [7]:
#Loop through
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    coordinates = str(row["Latitude"]) + "," + str(row["Longitude"])
    radius = 5000
    type = "lodging"
    key = g_key
    
    parameters = {
        "location": coordinates,
        "radius": radius,
        "type": type,
        "key": key,
    }
       
    response = requests.get(base_url, params=parameters).json()
       
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

Missing field/result... skipping.


,city,Country,Latitude,Longitude,Hotel Name
76,ituiutaba,BR,-18.9689,-49.4650,Bernal Hotel
182,lucapa,AO,-8.4192,20.7447,Residência Ulisses Dos Santos
278,mandalgovi,MN,45.7625,106.2708,В.Ганбат
283,komsomolskiy,UZ,40.4272,71.7189,Ikathouse
420,mount isa,AU,-20.7333,139.5000,ibis Styles Mt Isa Verona
448,shahrud,IR,36.4182,54.9763,Royal Hotel
485,barra do garcas,BR,-15.8900,-52.2567,Hotel Pousada Tropical
493,srednekolymsk,RU,67.4500,153.6833,


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))